In [179]:
import os
import json
import pandas as pd
import traceback
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain


In [180]:
from langchain.chat_models import ChatOpenAI

In [181]:
KEY = os.getenv("OPEN_AI_KEY")

In [182]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [183]:
TEMPLATE = """
  Text:{text}
  You are an expert MCQ maker. Given the above text, it is your job to\
  create a quiz of {number} multiple choice questions(MCQ) of a {subject}. In a {tone} tone.\
  Make sure the questions are not repeated and check all the questions to be conforming the text as well.
  Make sure to format your responses like RESPONSE_JSON below and use it as a guide.\
  Ensure to make {number} of MCQs
  {response_json}
"""

In [184]:
RESPONSE_JSON = {
  "1":{
    "mcq":"multiple choice question",
    "options":{
      "A": " choice here",
      "B": " choice here",
      "C": " choice here",
      "D": " choice here"
    },
    "correct":" correct answer letter"
    },
  "2":{
    "mcq":"multiple choice question",
    "options":{
      "A": " choice here",
      "B": " choice here",
      "C": " choice here",
      "D": " choice here"
    },
    "correct":" correct answer letter"
    },
  "3":{
    "mcq":"multiple choice question letter",
    "options":{
      "A": " choice here",
      "B": " choice here",
      "C": " choice here",
      "D": " choice here"
  },
    "correct":" correct answer"
  }
}

In [185]:
quiz_generation_prompt = PromptTemplate(
  input_variables = ["text","number","subject","tone","response_json"],
  template=TEMPLATE
)

In [186]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [187]:
TEMPLATE2 = """
  you are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} for students\
  You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis,
  if the quiz is not at per with the cognitive and analytical abilities of the students,\
  update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student.
  Quiz_MCQs:
  {quiz}
  
  Check from and expert English Writer of the above quiz:
"""

In [188]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [189]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [190]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables = ["text","number","subject","tone","response_json"],
                                          output_variables = ["quiz","review"], verbose=True)

In [191]:
file_path = r"C:\Users\user\Documents\Github\projects\openAi-langchain\mcqGenerator-project\data.txt"

In [192]:
file_path

'C:\\Users\\user\\Documents\\Github\\projects\\openAi-langchain\\mcqGenerator-project\\data.txt'

In [193]:
with open(file_path,"r") as file:
  TEXT = file.read()

In [194]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

In [195]:
NUMBER=5,
SUBJECT = "machine learning",
TONE ="simple"

In [196]:
with get_openai_callback() as cb:
  response=generate_evaluate_chain(
    {
      "text":TEXT,
      "number":NUMBER,
      "subject":SUBJECT,
      "tone":TONE,
      "response_json":json.dumps(RESPONSE_JSON)
    }
  )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field o

In [197]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 2088
Prompt Tokens: 1637
Completion Tokens: 451
Total Cost: 0.0033575


In [198]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [199]:
quiz = json.loads(response.get("quiz"))

In [200]:
type(quiz)

dict

In [201]:
quiz_table_data = []
for key,value in quiz.items():
  mcq = value["mcq"]
  options = " | ".join (
    [
      f"{option}:{option_value}"
      for option, option_value in value["options"].items()
    ] )
  correct = value["correct"]
  quiz_table_data.append({"MCQ":mcq,"Choices":options, "Correct":correct})

In [202]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'A:A field of study in artificial intelligence concerned with the development and study of statistical algorithms | B:A field of study in robotics concerned with developing physical machines | C:A field of study in astronomy concerned with studying celestial bodies | D:A field of study in biology concerned with plant genetics',
  'Correct': 'A'},
 {'MCQ': 'What is one of the applications of machine learning?',
  'Choices': 'A:Cooking recipes | B:Natural language processing | C:Car maintenance | D:Gardening tips',
  'Correct': 'B'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'A:Donald Hebb | B:Raytheon Company | C:Arthur Samuel | D:Tom M. Mitchell',
  'Correct': 'C'},
 {'MCQ': 'What is the mathematical foundation of machine learning?',
  'Choices': 'A:Algebra | B:Geometry | C:Mathematical optimization | D:Trigonometry',
  'Correct': 'C'},
 {'MCQ': 'What is the purpose of modern-day machine learning?',
  'Choices': 

In [203]:
quiz = pd.DataFrame(quiz_table_data)

In [204]:
quiz.to_csv("machinelearning.csv",index=False)